# Let's do something with data from the Eurovision Song Contest

Because why not

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import uuid
import datetime

In [62]:
# CONSTANTS

BASE_URL = 'https://eurovision.tv'
TODAY = datetime.datetime.now().strftime('%Y%m%d')

In [4]:
# preliminary test – are we getting data right away?
response = requests.get('https://eurovision.tv/event/lisbon-2018/grand-final'
                        '/participants').text

soup = BeautifulSoup(response, 'lxml')

table_rows = soup.select('.row__TableRow-an9049-0')

for row in table_rows:
  print(row.text)
  

R/OCountryArtistsongPtsPointsRankRanking
01UkraineMELOVINUnder The Ladder13017
02SpainAmaia y AlfredTu Canción6123
03SloveniaLea SirkHvala, ne!6422
04LithuaniaIeva ZasimauskaitėWhen We're Old18112
05AustriaCesár SampsonNobody But You3423
06EstoniaElina NechayevaLa Forza2458
07NorwayAlexander RybakThat's How You Write A Song14415
08PortugalCláudia PascoalO Jardim3926
09United KingdomSuRieStorm4824
10SerbiaSanja Ilić & BalkanikaNova Deca11319
11GermanyMichael SchulteYou Let Me Walk Alone3404
12AlbaniaEugent BushpepaMall18411
13FranceMadame MonsieurMercy17313
14Czech RepublicMikolas JosefLie To Me2816
15DenmarkRasmussenHigher Ground2269
16AustraliaJessica MauboyWe Got Love9920
17FinlandSaara AaltoMonsters4625
18BulgariaEQUINOXBones16614
19MoldovaDoReDoSMy Lucky Day20910
20SwedenBenjamin IngrossoDance You Off2747
21HungaryAWSViszlát Nyár9321
22IsraelNettaTOY5291
23The NetherlandsWaylonOutlaw In 'Em12118
24IrelandRyan O'ShaughnessyTogether13616
25CyprusEleni FoureiraFuego4362
26ItalyErmal M

In [5]:
# Cool, so this works, which is nice – since that means that the data is 
# actually prerendered on the server and not generated by javascript in the 
# browser – although the way the website behaves certainly suggests so.

## Data Model

This should be the fully realised data model:

![Data Model](super-extended-model.svg)

Depending on the time, some of these tables might not exist and/or are already part of the main `song`table. 

At this point of time I'm not honing in onto a specific question, but try to keep an open mind what the collected data might reveal.

## Contests

In [9]:
# Get all the contests from the entry page
contests_response = requests.get('https://eurovision.tv/events').text
contests_soup = BeautifulSoup(contests_response, 'lxml')


In [10]:

contests_elements = contests_soup.select('.result__Wrapper-s9upcfm-1')

contests = []

for contest_element in contests_elements:
  name = ' '.join(contest_element.select_one(
    '.text__Text-x6y36f-0').stripped_strings)
  location = re.search(r'^(.*) \d{4}', name).group(1)
  year = re.search(r'\d{4}', name).group(0)
  
  contest = {
    'url': contest_element['href'],
    'name': name,
    # Let's get the image, too, just for kicks
    'img_src': contest_element.select_one('img')['src'],
    'year': year,
    'location': location,
    'id': uuid.uuid3(uuid.NAMESPACE_URL, BASE_URL + contest_element['href'])
  }
  
  contests.append(contest)
  
contests_df = pd.DataFrame(contests)



In [6]:
contests_df

,id,img_src,location,name,url,year
0,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,https://apex.eurovision.tv/image/52219b044b79f...,Lisbon,Lisbon 2018,/event/lisbon-2018,2018
1,42e0141d-3b05-3776-ba39-ea69c28e8bdb,https://apex.eurovision.tv/image/7b0be5f440549...,Kyiv,Kyiv 2017,/event/kyiv-2017,2017
2,9b967659-2937-3ddc-97a8-7d4296ecdc2b,https://apex.eurovision.tv/image/9ef19ed95f0ce...,Stockholm,Stockholm 2016,/event/stockholm-2016,2016
3,dcd86fee-f06a-315a-abcf-372a8fba2e7f,https://apex.eurovision.tv/image/6b2a4289be78d...,Vienna,Vienna 2015,/event/vienna-2015,2015
4,65ff473b-91bc-335f-bed3-9052fc7cdeb5,https://apex.eurovision.tv/image/a7c9ca682b37e...,Copenhagen,Copenhagen 2014,/event/copenhagen-2014,2014
5,fc7505da-0220-34ba-82a1-3abc5e8b3e36,https://apex.eurovision.tv/image/acf1012f98180...,Malmö,Malmö 2013,/event/malmo-2013,2013
6,1e72efe5-71b9-3669-9b83-b0857549f15b,https://apex.eurovision.tv/image/de26c67acd021...,Baku,Baku 2012,/event/baku-2012,2012
7,888687e1-8da0-3d6c-ab5a-ded1f7e9ac21,https://apex.eurovision.tv/image/06cbdc0fccf0e...,Düsseldorf,Düsseldorf 2011,/event/dusseldorf-2011,2011
8,459089e7-dc8b-3af3-816c-e195d8a63533,https://apex.eurovision.tv/image/2ef4dd5de1f60...,Oslo,Oslo 2010,/event/oslo-2010,2010
9,fa0099a0-6a5e-3a2e-913f-5e19be93c801,https://apex.eurovision.tv/image/a1834353e3c7e...,Moscow,Moscow 2009,/event/moscow-2009,2009


In [11]:
# save to csv
contests_df.to_csv('project/contests-' + TODAY + '.csv', index=False)

## Finalists

In [ ]:
# Challenges: at some point during the contest, the format changed from just 
# having a final to having two semi-finals and a grand final – unfortunately,
#  this also means that the URL structure changes there. This certainly does 
# not make me that happy.


In [ ]:

from time import sleep
from random import randint


In [39]:
# this should help me keep track of the requests and give me an idea when I'm
#  starting to tax the system too much
request_stats = []

In [41]:
def get_song_details(song_url):
  """
  Download song details from the detail page on eurovision.tv. This mainly 
  consists of the broadcasting company, the writers and composers.
  
  :param song_url: Unique URL fragment for the song
  :return: A dictionary containing the additional data
  """
  full_url = BASE_URL + song_url

  # Let's be cautious and not pelt the server with too many requests at once
  delay = randint(1, 20)
  sleep(delay)
  response = requests.get(full_url)

  if response.ok is not True:
    print('  Request for {} failed.'.format(song_url))
    print('  Status:', response.status_code)
    return {}
  
  #  Collect data for request statistics
  request_stat = {
    'delay': delay,
    'url': response.url,
    'status': response.status_code,
    'timestamp': datetime.datetime.now().timestamp()
  }
  
  request_stats.append(request_stat)
  
  # Get data out
  song_soup = BeautifulSoup(response.text, 'lxml')

  metadata_element = song_soup.select_one('[class*="songDetails"] + div')
  info_box_elements = metadata_element.select('div[class*="column"]')

  # Order of infoboxes:
  # - title
  # - performed by
  # - written by
  # - composed by
  # - broadcaster

  song_details = {
    'writers': info_box_elements[2].select_one('dd').text,
    'composers': info_box_elements[3].select_one('dd').text,
    'broadcaster': info_box_elements[4].select_one('dd').text
  }

  return song_details


In [38]:
get_song_details('/participant/imri')

{'broadcaster': 'IBA',
 'composers': 'Dolev Ram, Penn Hazut',
 'writers': 'Dolev Ram, Penn Hazut'}

In [3]:
def get_finalists(contest_url_fragement):
  """
  Function to get all contestants that reached a/the final.
  
  :param contest_url_fragement: URL fragment defining the contest
  :return: A list of all the finalists of this year
  """
  
  full_url = BASE_URL + contest_url_fragement + '/final'

  response = requests.get(full_url)
  final_type = 'final'

  # Whoops, nothing to see at "/final". Let's try "/grand-final" instead
  if 200 < response.status_code < 500:
    full_url = BASE_URL + contest_url_fragement + '/grand-final'
    response = requests.get(full_url)
    final_type = 'grand final'

  contest_soup = BeautifulSoup(response.text, 'lxml')

  header_cells = contest_soup.select('table thead tr th '
                                     '.small-caps__SmallCaps-s1ooca2g-0')
  headers = [cell.text for cell in header_cells]

  finalists_rows = contest_soup.select('table tbody tr')

  finalists = []

  for finalist in finalists_rows:
    
    # get all cells in the row
    finalists_cells = finalist.select('td')
    
    # get some special cells, like the entry URL (which, strangely enough, 
    # is connected with the artist, and not the song
    song_url = finalists_cells[2].select_one('a')['href']
    
    # Set up the song entry
    entry = {
      'contest_id': uuid.uuid3(uuid.NAMESPACE_URL, BASE_URL + 
                               contest_url_fragement),
      'final_type': final_type,
      'url': song_url,
      'id': uuid.uuid3(uuid.NAMESPACE_URL, BASE_URL + song_url)
    }
    
    # go through the table row and add data to the dictionary
    for i in range(len(finalist)):
      entry[headers[i]] = finalists_cells[i].text
    
    # Find more data on the individual song page
    entry.update(get_song_details(song_url))
    
    #TODO Connect to last.fm API and look for even more data, because why not

    finalists.append(entry)

  return finalists


In [31]:
# testing, testing ...
get_finalists('/event/bergen-1986')

5


5


5


5


5


...

5


5


5


5


[{'Artist': 'Sherisse Laurence',
  'Country': 'Luxembourg',
  'PtsPoints': '117',
  'R/O': '01',
  'RankRanking': '3',
  'broadcaster': 'CLT',
  'composers': 'Rolf Soja',
  'contest_id': UUID('0ee2efac-f244-3b1d-894f-7d1ebfb873ec'),
  'final_type': 'final',
  'id': UUID('51d675a7-dd90-3df7-91f0-f504cf032b54'),
  'song': "L'amour De Ma Vie",
  'url': '/participant/sherisse-laurence',
  'writers': 'Alain Garcia, Frank Dostal'},
 {'Artist': 'Doris Dragovic',
  'Country': 'Yugoslavia',
  'PtsPoints': '49',
  'R/O': '02',
  'RankRanking': '11',
  'broadcaster': 'JRT',
  'composers': 'Zrinko Tutic',
  'contest_id': UUID('0ee2efac-f244-3b1d-894f-7d1ebfb873ec'),
  'final_type': 'final',
  'id': UUID('b022a4bb-3398-3a44-940d-417edcb1c428'),
  'song': 'Zeljo Moja',
  'url': '/participant/doris-dragovic',
  'writers': '-'},
 {'Artist': 'Cocktail Chic',
  'Country': 'France',
  'PtsPoints': '13',
  'R/O': '03',
  'RankRanking': '17',
  'broadcaster': 'A2F',
  'composers': 'Georges Costa, Michel Co

In [12]:
# Okay, I guess we're just going to loop over the url fragments and use that 
# to create a new dataframe with all the contest entries that reached the final

finalists_list = []

counter = 0
for url in contests_df['url']:
  print('Get contest at', url)
  print('Index:', counter)
  sleep(randint(1,10))
  finalists_list = finalists_list + get_finalists(url)
  counter = counter + 1


In [16]:

finalists_df = pd.DataFrame(finalists_list)


In [17]:
finalists_df.to_csv('project/finalists-' + TODAY + '.csv', index=False)

In [25]:
finalists_df.dtypes


In [26]:
# use the url as the index as this is more useful that just numbers
finalists_df.set_index('url', inplace=True, verify_integrity=True)


In [27]:
# get the lines where the requests failed and we have missing data
missing_data_slice = finalists_df[finalists_df['broadcaster'].isna()]
missing_data_slice

,Artist,Country,PtsPoints,R/O,RankRanking,broadcaster,composers,contest_id,final_type,id,song,writers
url,,,,,,,,,,,,
/participant/loreen,Loreen,Sweden,372,17,1,NaN,NaN,1e72efe5-71b9-3669-9b83-b0857549f15b,grand final,094283a4-43c8-397c-9c16-5285fa1bfbfd,Euphoria,NaN
/participant/roman-lob,Roman Lob,Germany,110,20,8,NaN,NaN,1e72efe5-71b9-3669-9b83-b0857549f15b,grand final,3bf3af89-4333-3f48-b0fb-816a0dd30853,Standing Still,NaN
/participant/gaitana,Gaitana,Ukraine,65,25,15,NaN,NaN,1e72efe5-71b9-3669-9b83-b0857549f15b,grand final,6ca48fd5-515a-3407-b351-7364c70f926c,Be My Guest,NaN
/participant/pasha-parfeny,Pasha Parfeny,Moldova,81,26,11,NaN,NaN,1e72efe5-71b9-3669-9b83-b0857549f15b,grand final,44f53f3c-702c-3561-9bce-12ca2e773300,Lăutar,NaN
/participant/dervish,Dervish,Ireland,5,04,24,NaN,NaN,e4ca4b05-c62d-3bc9-844c-2711746e7694,grand final,466f1c90-b973-3240-98ea-aa0e1342af66,They Can't Stop The Spring,NaN
/participant/hanna-pakarinen,Hanna Pakarinen,Finland,53,05,17,NaN,NaN,e4ca4b05-c62d-3bc9-844c-2711746e7694,grand final,f86cf4ed-186e-34bc-bc35-fc16793d1a8a,Leave Me Alone,NaN
/participant/alenka-gotar,Alenka Gotar,Slovenia,66,07,15,NaN,NaN,e4ca4b05-c62d-3bc9-844c-2711746e7694,grand final,a08342cd-3e72-3c3a-baaa-b26faceb2089,Cvet Z Juga,NaN
/participant/sarbel,Sarbel,Greece,139,10,7,NaN,NaN,e4ca4b05-c62d-3bc9-844c-2711746e7694,grand final,bb7b656e-7005-35da-be7e-deb95caf35ea,Yassou Maria,NaN
/participant/sopho,Sopho,Georgia,97,11,12,NaN,NaN,e4ca4b05-c62d-3bc9-844c-2711746e7694,grand final,9c0872c8-4e3a-3034-aab8-091e96100529,Visionary Dream,NaN


In [65]:
def fill_missing_pieces(finalist_row):
  """
  This function will use the row with missing data and request the data 
  again, filling in the blanks.
  
  :param finalist_row: Series
  :return: 
  """
  # get the url
  url = finalist_row.name
  
  # execute the finalists data retrieval function
  details = get_song_details(url)

  # use df.at[] to place data at the correct place
  for detail_tuple in details.items():
    finalists_df.at[url, detail_tuple[0]] = detail_tuple[1]
    

In [66]:
missing_data_slice.apply(fill_missing_pieces, axis=1)

  Request for /participant/loreen failed.
  Status: 502


  Request for /participant/gaitana failed.
  Status: 502


  Request for /participant/dervish failed.
  Status: 502


url
/participant/loreen                            None
/participant/roman-lob                         None
/participant/gaitana                           None
/participant/pasha-parfeny                     None
/participant/dervish                           None
/participant/hanna-pakarinen                   None
/participant/alenka-gotar                      None
/participant/sarbel                            None
/participant/sopho                             None
/participant/the-ark                           None
/participant/les-fatals-picards                None
/participant/lynn-chirchop                     None
/participant/mija-martina                      None
/participant/rita-guerra                       None
/participant/lou                               None
/participant/t-a-t-u                           None
/participant/beth                              None
/participant/jemini                            None
/participant/olexandr                          None
/partici

In [67]:
# is it better now?
finalists_df[finalists_df['broadcaster'].isna()]

,Artist,Country,PtsPoints,R/O,RankRanking,broadcaster,composers,contest_id,final_type,id,song,writers
url,,,,,,,,,,,,
/participant/loreen,Loreen,Sweden,372,17,1,NaN,NaN,1e72efe5-71b9-3669-9b83-b0857549f15b,grand final,094283a4-43c8-397c-9c16-5285fa1bfbfd,Euphoria,NaN
/participant/gaitana,Gaitana,Ukraine,65,25,15,NaN,NaN,1e72efe5-71b9-3669-9b83-b0857549f15b,grand final,6ca48fd5-515a-3407-b351-7364c70f926c,Be My Guest,NaN
/participant/dervish,Dervish,Ireland,5,04,24,NaN,NaN,e4ca4b05-c62d-3bc9-844c-2711746e7694,grand final,466f1c90-b973-3240-98ea-aa0e1342af66,They Can't Stop The Spring,NaN


In [69]:
# ... and again

finalists_df[finalists_df['broadcaster'].isna()].apply(fill_missing_pieces, 
                                                       axis=1)

url
/participant/gaitana    None
/participant/dervish    None
dtype: object

In [70]:
# save the final data frame
finalists_df.to_csv('project/finalists-' + TODAY + '.csv', index=False)

In [71]:
# Have some data:
finalists_df

,Artist,Country,PtsPoints,R/O,RankRanking,broadcaster,composers,contest_id,final_type,id,song,writers
url,,,,,,,,,,,,
/participant/melovin,MELOVIN,Ukraine,130,01,17,UAPBC,MELOVIN,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,459bf9d8-4498-3fbe-9e2f-951162e4315b,Under The Ladder,Mike Ryals
/participant/alfred-amaia,Amaia y Alfred,Spain,61,02,23,TVE,"Raul Gomez Garcia, Sylvia Ruth Santoro Lopez",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,9c67c117-97fa-3f89-8fec-11d2d608f684,Tu Canción,"Raul Gomez Garcia, Sylvia Ruth Santoro Lopez"
/participant/lea-sirk,Lea Sirk,Slovenia,64,03,22,RTVSLO,"Lea Sirk, Tomy DeClerque",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,e2fbabfe-1daf-343e-82c2-57fb62261db3,"Hvala, ne!",Lea Sirk
/participant/Ieva-zasimauskaite,Ieva Zasimauskaitė,Lithuania,181,04,12,LRT,Vytautas Bikus,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,c76ba2e9-4ea1-3789-ba07-a8bc519edb04,When We're Old,Vytautas Bikus
/participant/cesar-sampson,Cesár Sampson,Austria,342,05,3,ORF,"Sebastian Arman, Cesár Sampson, Joacim Persson...",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,38f9258f-8955-3447-ad21-93136cf82a1d,Nobody But You,"Sebastian Arman, Cesár Sampson, Joacim Persson..."
/participant/elina-nechayeva,Elina Nechayeva,Estonia,245,06,8,ERR,"Mihkel Mattisen, Timo Vendt",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,5eca6de2-673f-30a6-8ebe-8757ac3583cd,La Forza,"Elina Nechayeva, Ksenia Kuchukova"
/participant/alexander-rybak-2018,Alexander Rybak,Norway,144,07,15,NRK,Alexander Rybak,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,61722c77-cecd-31ac-ab91-93b9c12fec13,That's How You Write A Song,Alexander Rybak
/participant/claudia-pascoal,Cláudia Pascoal,Portugal,39,08,26,RTP,Isaura,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,55d64e52-3a7f-3a5a-8b9f-9e9653e618b2,O Jardim,Isaura
/participant/surie,SuRie,United Kingdom,48,09,24,BBC,"Nicole Blair, Gil Lewis, Sean Hargreaves",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,92c702fc-cbbc-38de-a57e-d7a3fae5fe6d,Storm,"Nicole Blair, Gil Lewis, Sean Hargreaves"


## Stuff left to do

- Separate the individuals in the writers and composers column and move them to their own tables
- Write last.fm API requests to get the tags for the individual songs; clean up and normalise the tags
- Use the tags to see whether certain styles/tags are more prevalent in certain countries than others
- Generally analysing the data to see whether there are interesting things to find

In [7]:
# Well, of *course* by now the notebook forgot all about the variables I 
# previously used, so I have to re-import the table again.

finalists_df = pd.read_csv('finalists-20180629.csv')
finalists_df.astype({'id': uuid.UUID})

,Artist,Country,PtsPoints,R/O,RankRanking,broadcaster,composers,contest_id,final_type,id,song,writers
0,MELOVIN,Ukraine,130,1,17,UAPBC,MELOVIN,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,459bf9d8-4498-3fbe-9e2f-951162e4315b,Under The Ladder,Mike Ryals
1,Amaia y Alfred,Spain,61,2,23,TVE,"Raul Gomez Garcia, Sylvia Ruth Santoro Lopez",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,9c67c117-97fa-3f89-8fec-11d2d608f684,Tu Canción,"Raul Gomez Garcia, Sylvia Ruth Santoro Lopez"
2,Lea Sirk,Slovenia,64,3,22,RTVSLO,"Lea Sirk, Tomy DeClerque",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,e2fbabfe-1daf-343e-82c2-57fb62261db3,"Hvala, ne!",Lea Sirk
3,Ieva Zasimauskaitė,Lithuania,181,4,12,LRT,Vytautas Bikus,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,c76ba2e9-4ea1-3789-ba07-a8bc519edb04,When We're Old,Vytautas Bikus
4,Cesár Sampson,Austria,342,5,3,ORF,"Sebastian Arman, Cesár Sampson, Joacim Persson...",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,38f9258f-8955-3447-ad21-93136cf82a1d,Nobody But You,"Sebastian Arman, Cesár Sampson, Joacim Persson..."
5,Elina Nechayeva,Estonia,245,6,8,ERR,"Mihkel Mattisen, Timo Vendt",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,5eca6de2-673f-30a6-8ebe-8757ac3583cd,La Forza,"Elina Nechayeva, Ksenia Kuchukova"
6,Alexander Rybak,Norway,144,7,15,NRK,Alexander Rybak,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,61722c77-cecd-31ac-ab91-93b9c12fec13,That's How You Write A Song,Alexander Rybak
7,Cláudia Pascoal,Portugal,39,8,26,RTP,Isaura,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,55d64e52-3a7f-3a5a-8b9f-9e9653e618b2,O Jardim,Isaura
8,SuRie,United Kingdom,48,9,24,BBC,"Nicole Blair, Gil Lewis, Sean Hargreaves",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,92c702fc-cbbc-38de-a57e-d7a3fae5fe6d,Storm,"Nicole Blair, Gil Lewis, Sean Hargreaves"
9,Sanja Ilić & Balkanika,Serbia,113,10,19,RTS,"Aleksandar Sanja Ilić, Tatjana Karajanov Ilić",c7aae442-7c52-3a14-9bf5-2fb1880fbf58,grand final,0d0d6074-cca7-3e1c-96f5-a71be127cf3a,Nova Deca,Danica Krstajić


In [14]:
finalists_df.shape

(1196, 12)

In [30]:
finalists_df['id'].dtype

dtype('O')

## Extraction of the Performers into their own table

In [28]:
from slugify import slugify

def normalise_artist_names(name):
  # We're just gonna use slugify to normalise the artist name and compare 
  # against that
  return slugify(name.strip())


In [7]:
def get_uuid_for_artist(artist_name):
  PREFIX = 'internal://artist/'
  return uuid.uuid3(
    uuid.NAMESPACE_URL,
    PREFIX + normalise_artist_names(artist_name))


In [3]:

normalise_artist_names('Kasia Moś')


In [6]:
normalise_artist_names('  Izel Çeliköz, Rayhan Soykarçi and Can Ugurluér ')

'izel-celikoz-rayhan-soykarci-and-can-ugurluer'

In [4]:
normalise_artist_names('Gabriela Gunčíková')

'gabriela-guncikova'

In [11]:
get_uuid_for_artist('  Izel Çeliköz, Rayhan Soykarçi and Can Ugurluér ') == get_uuid_for_artist('Izel Çeliköz Rayhan Soykarçi and Can Ugurluér')

True

In [85]:
# create empty lists
# use a dict for artists, so we can "search" by UUID
artists_list = {}

# use normal lists for the artist-song-connections
performers = []
writers = []
composers = []

In [37]:

def add_artist_to_lists(song_id, artist_name, artists_table, join_table):
  """
  Normalise the artist name, generate a UUID, check whether it is already 
  part of the artists list. Add it to the list if not, otherwise just add to 
  the set of known aliases (not sure if that is necessary, but ... let's do 
  it anyway).
  
  Then add the artist UUID and song UUID to the respective join table (
  performer, writter, composer, etc)
  
  :param song_id: The ID of the song
  :param artist_name: The name of the artist, as gathered from the website
  :param artists_table: The big list of all the artists
  :param join_table: The table to add the song/artist connection to.
  :return: 
  """
  # generate the uuid of the artist
  artist_id = get_uuid_for_artist(artist_name)

  # check if the uuid already exists in the artist list
  if artist_id in artists_table:
    # if yes, add to the set of aliases of the specific artist
    artists_table[artist_id]['aliases'].add(artist_name)

  else:
    # if no, add to the list of artists
    artists_table[artist_id] = {
      'name': artist_name,
      'normalised_name': normalise_artist_names(artist_name),
      'aliases': {artist_name}
    }

  # add song UUID and artist UUID to performers list
  join_table.append(
    {
      'song_id': song_id,
      'artist_id': artist_id
    }
  )


In [69]:
def split_creators(creators):
  if creators == '-':
    return []
  
  pattern = r'\s*[,/&+()]+\s*|\Wand\W'
  splitted = re.split(pattern, creators)
  return splitted

In [72]:
split_creators('-')

[]

In [73]:
split_creators('Christos Dantis & Natalia Germanou')

['Christos Dantis', 'Natalia Germanou']

In [74]:
split_creators('Marie-Line Marolany')

['Marie-Line Marolany']

In [75]:
split_creators('Krisztina Bokor Fekete/Attila Kornyei')

['Krisztina Bokor Fekete', 'Attila Kornyei']

In [76]:
split_creators("Moshe Datz, Gabriel Batler, Yaacov Lymay and Jacky Oved")

['Moshe Datz', 'Gabriel Batler', 'Yaacov Lymay', 'Jacky Oved']

In [77]:
split_creators("DNA (David Musumeci & Anthony Egizii), Michael Angelo")

['DNA', 'David Musumeci', 'Anthony Egizii', 'Michael Angelo']

In [78]:
def extract_artists(row):
  """
  Apply function that extracts all artists of a song.
  :param row: 
  :return: 
  """
  
  # Get song performer
  add_artist_to_lists(row['id'], row['Artist'], artists_list, performers)
  
  # Get composers
  for composer in split_creators(row['composers']):
    add_artist_to_lists(row['id'], composer, artists_list, composers)
    
  # Get Writers
  for writer in split_creators(row['writers']):
    add_artist_to_lists(row['id'], writer, artists_list, writers)

In [79]:
finalists_df.head(5).apply(extract_artists, axis=1)

0    None
1    None
2    None
3    None
4    None
dtype: object

In [80]:
performers

[{'artist_id': UUID('ffbc56f4-465c-3b95-81d9-48b2c1b58f13'),
  'song_id': '459bf9d8-4498-3fbe-9e2f-951162e4315b'},
 {'artist_id': UUID('e1f46057-0ab5-31b3-9420-65cf623f8bf1'),
  'song_id': '9c67c117-97fa-3f89-8fec-11d2d608f684'},
 {'artist_id': UUID('7dbc0493-bf4c-3bf3-89d9-2f05f6d9a104'),
  'song_id': 'e2fbabfe-1daf-343e-82c2-57fb62261db3'},
 {'artist_id': UUID('e4f3b762-9e21-3017-9963-445ad881b03b'),
  'song_id': 'c76ba2e9-4ea1-3789-ba07-a8bc519edb04'},
 {'artist_id': UUID('6878ff53-7b2a-394f-9603-bcc6fd90817a'),
  'song_id': '38f9258f-8955-3447-ad21-93136cf82a1d'},
 {'artist_id': UUID('ffbc56f4-465c-3b95-81d9-48b2c1b58f13'),
  'song_id': '459bf9d8-4498-3fbe-9e2f-951162e4315b'},
 {'artist_id': UUID('e1f46057-0ab5-31b3-9420-65cf623f8bf1'),
  'song_id': '9c67c117-97fa-3f89-8fec-11d2d608f684'},
 {'artist_id': UUID('7dbc0493-bf4c-3bf3-89d9-2f05f6d9a104'),
  'song_id': 'e2fbabfe-1daf-343e-82c2-57fb62261db3'},
 {'artist_id': UUID('e4f3b762-9e21-3017-9963-445ad881b03b'),
  'song_id': 'c76ba

In [81]:
artists_list

{UUID('124f1ae7-cafe-3b36-a73a-acca5ae80275'): {'aliases': {'Sylvia Ruth Santoro Lopez'},
  'name': 'Sylvia Ruth Santoro Lopez',
  'normalised_name': 'sylvia-ruth-santoro-lopez'},
 UUID('406ae149-e048-3e2b-8ecd-ccca890e71f1'): {'aliases': {'Sebastian Arman'},
  'name': 'Sebastian Arman',
  'normalised_name': 'sebastian-arman'},
 UUID('5bf6004a-10ae-397b-ae53-c09958ab31fd'): {'aliases': {'Borislav Milanov'},
  'name': 'Borislav Milanov',
  'normalised_name': 'borislav-milanov'},
 UUID('63ad2777-a81b-3fa2-83a0-e29655f4e2b0'): {'aliases': {'Mike Ryals'},
  'name': 'Mike Ryals',
  'normalised_name': 'mike-ryals'},
 UUID('6878ff53-7b2a-394f-9603-bcc6fd90817a'): {'aliases': {'Cesár Sampson'},
  'name': 'Cesár Sampson',
  'normalised_name': 'cesar-sampson'},
 UUID('77250a06-608a-3f03-9b4a-c1c0cb05c711'): {'aliases': {'Raul Gomez Garcia'},
  'name': 'Raul Gomez Garcia',
  'normalised_name': 'raul-gomez-garcia'},
 UUID('7dbc0493-bf4c-3bf3-89d9-2f05f6d9a104'): {'aliases': {'Lea Sirk'},
  'name':

In [82]:
writers

[{'artist_id': UUID('63ad2777-a81b-3fa2-83a0-e29655f4e2b0'),
  'song_id': '459bf9d8-4498-3fbe-9e2f-951162e4315b'},
 {'artist_id': UUID('77250a06-608a-3f03-9b4a-c1c0cb05c711'),
  'song_id': '9c67c117-97fa-3f89-8fec-11d2d608f684'},
 {'artist_id': UUID('124f1ae7-cafe-3b36-a73a-acca5ae80275'),
  'song_id': '9c67c117-97fa-3f89-8fec-11d2d608f684'},
 {'artist_id': UUID('7dbc0493-bf4c-3bf3-89d9-2f05f6d9a104'),
  'song_id': 'e2fbabfe-1daf-343e-82c2-57fb62261db3'},
 {'artist_id': UUID('7e4ff47d-b059-39d1-b576-445ad7344b7c'),
  'song_id': 'c76ba2e9-4ea1-3789-ba07-a8bc519edb04'},
 {'artist_id': UUID('406ae149-e048-3e2b-8ecd-ccca890e71f1'),
  'song_id': '38f9258f-8955-3447-ad21-93136cf82a1d'},
 {'artist_id': UUID('6878ff53-7b2a-394f-9603-bcc6fd90817a'),
  'song_id': '38f9258f-8955-3447-ad21-93136cf82a1d'},
 {'artist_id': UUID('e7dba6ab-32dd-3e5a-afd6-d34ccd5314b2'),
  'song_id': '38f9258f-8955-3447-ad21-93136cf82a1d'},
 {'artist_id': UUID('95ce1a17-29f5-3ff5-9b9d-e10f43e97f27'),
  'song_id': '38f92

In [83]:
composers

[{'artist_id': UUID('ffbc56f4-465c-3b95-81d9-48b2c1b58f13'),
  'song_id': '459bf9d8-4498-3fbe-9e2f-951162e4315b'},
 {'artist_id': UUID('77250a06-608a-3f03-9b4a-c1c0cb05c711'),
  'song_id': '9c67c117-97fa-3f89-8fec-11d2d608f684'},
 {'artist_id': UUID('124f1ae7-cafe-3b36-a73a-acca5ae80275'),
  'song_id': '9c67c117-97fa-3f89-8fec-11d2d608f684'},
 {'artist_id': UUID('7dbc0493-bf4c-3bf3-89d9-2f05f6d9a104'),
  'song_id': 'e2fbabfe-1daf-343e-82c2-57fb62261db3'},
 {'artist_id': UUID('ac9a30ac-ef18-3aed-acb6-76559a9ef2ad'),
  'song_id': 'e2fbabfe-1daf-343e-82c2-57fb62261db3'},
 {'artist_id': UUID('7e4ff47d-b059-39d1-b576-445ad7344b7c'),
  'song_id': 'c76ba2e9-4ea1-3789-ba07-a8bc519edb04'},
 {'artist_id': UUID('406ae149-e048-3e2b-8ecd-ccca890e71f1'),
  'song_id': '38f9258f-8955-3447-ad21-93136cf82a1d'},
 {'artist_id': UUID('6878ff53-7b2a-394f-9603-bcc6fd90817a'),
  'song_id': '38f9258f-8955-3447-ad21-93136cf82a1d'},
 {'artist_id': UUID('e7dba6ab-32dd-3e5a-afd6-d34ccd5314b2'),
  'song_id': '38f92

In [86]:
finalists_df.apply(extract_artists, axis=1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1166    None
1167    None
1168    None
1169    None
1170    None
1171    None
1172    None
1173    None
1174    None
1175    None
1176    None
1177    None
1178    None
1179    None
1180    None
1181    None
1182    None
1183    None
1184    None
1185    None
1186    None
1187    None
1188    None
1189    None
1190    None
1191    None
1192    None
1193    None
1194    None
1195    None
Length: 1196, dtype: object

In [89]:
artists_df = pd.DataFrame.from_dict(artists_list, orient='index')

In [90]:
artists_df

,name,normalised_name,aliases
001897e6-1eba-3b65-b463-d93f2547ce03,Tooji,tooji,{Tooji}
0023cd10-d9bf-3eba-b6b3-f08431c478b5,Nurit Hirsh,nurit-hirsh,{Nurit Hirsh}
0050b349-6048-3832-8439-623cc083e16c,Jan van Dijck,jan-van-dijck,{Jan van Dijck}
006ac98e-5c6c-3e17-97c9-bb3309505437,ENI,eni,{ENI}
008ae253-bf9b-356e-a0e4-0c4280c55da2,Dragan Mijatovic,dragan-mijatovic,{Dragan Mijatovic}
009722f5-a35e-3542-bba4-06a4e7bf0b99,Jonatan Cerrada,jonatan-cerrada,{Jonatan Cerrada}
0099bd53-6bdd-3d9c-b47e-7e873e0e7160,Rick Allison,rick-allison,{Rick Allison}
00a0ac84-8ea7-3f90-a046-c6bcc5107978,Claude Henri Vic,claude-henri-vic,{Claude Henri Vic}
00a0ca50-034a-39d1-aac8-ff2aa2bde094,Fabio Ricci,fabio-ricci,{Fabio Ricci}
00ad84bf-eddc-30ea-91e8-d5f3fcdb3d4e,MFÖ,mfo,{MFÖ}


In [93]:
artists_df.to_csv('project/artists-' + TODAY + '.csv', index_label='id')

In [92]:
pd.DataFrame(performers).to_csv('project/performers-' + TODAY + '.csv', index=False)
pd.DataFrame(writers).to_csv('project/writers-' + TODAY + '.csv', 
                             index=False)
pd.DataFrame(composers).to_csv('project/composers-' + TODAY + '.csv', 
                               index=False)

## Let's get some additional data from Last.FM

In [1]:
LAST_FM_ENDPOINT = 'http://ws.audioscrobbler.com'
from apikeys import LAST_FM_APIKEY

In [52]:
from urllib.parse import urlencode

def get_last_fm_track_url(artist, song_name):
  query = {
    'method': 'track.getInfo',
    'api_key': LAST_FM_APIKEY,
    'artist': artist,
    'track': song_name,
    'format': 'json',
    'autocorrect': 1
  }
  url = '{}/2.0/?{}'.format(
    LAST_FM_ENDPOINT,
    urlencode(query)
  )
  
  return url


In [31]:
def normalise_tag_name(tag):
  return slugify(tag, regex_pattern=re.compile(r'[^-a-z]+'))

In [34]:
def get_uuid_for_tag(tag):
  PREFIX = 'internal://tag/'
  return uuid.uuid3(uuid.NAMESPACE_URL, PREFIX + normalise_tag_name(tag))

In [58]:
import time
from random import randint

def get_top_tags(artist, song_name):
  try:
    time.sleep(randint(2, 15))
    response = requests.get(get_last_fm_track_url(artist, song_name))
    track_data = response.json()
    if 'track' in track_data:
      tags = track_data['track']['toptags']['tag']
      return [tag['name'] for tag in tags]

    elif 'error' in track_data:
      print('  ', track_data['message'])
      return []

  except:
    print('    Some unknown error occurred.')
    return []



In [59]:
tags_list = {}
tags_connections = []
counter = 0
fullLength = len(finalists_df)

def collect_tags(row):
  print('{:>5.0%} | {:>5} | {}: {}'.format((row.name + 1) / fullLength, 
                                          row.name, 
                                  row['Artist'], row['song']))
  for tag in get_top_tags(row['Artist'], row['song']):
    id = get_uuid_for_tag(tag)

    if id in tags_list:
      tags_list[id]['aliases'].add(tag)

    else:
      tags_list[id] = {
        'name': normalise_tag_name(tag),
        'aliases': {tag}
      }

    tags_connections.append(
      {
        'song_id': row['id'],
        'tag_id': id
      }
    )


In [60]:
finalists_df.apply(collect_tags, axis=1)

   0% |     0 | MELOVIN: Under The Ladder


   0% |     1 | Amaia y Alfred: Tu Canción


   0% |     2 | Lea Sirk: Hvala, ne!


   0% |     3 | Ieva Zasimauskaitė: When We're Old


   0% |     4 | Cesár Sampson: Nobody But You


...

 100% |  1192 | Freddy Quinn: So Geht Das Jede Nacht


 100% |  1193 | Dany Dauberson: Il Est Là


 100% |  1194 | Michèle Arnaud: Les Amants De Minuit


 100% |  1195 | Tonina Torielli: Amami Se Vuoi


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1166    None
1167    None
1168    None
1169    None
1170    None
1171    None
1172    None
1173    None
1174    None
1175    None
1176    None
1177    None
1178    None
1179    None
1180    None
1181    None
1182    None
1183    None
1184    None
1185    None
1186    None
1187    None
1188    None
1189    None
1190    None
1191    None
1192    None
1193    None
1194    None
1195    None
Length: 1196, dtype: object

In [50]:
tags_list

{UUID('00733041-c806-362f-8ebd-5bc7359c7edf'): {'aliases': {'UK'},
  'name': 'uk'},
 UUID('01584a50-dd0c-3c24-95eb-f1b69c3b5696'): {'aliases': {'spain'},
  'name': 'spain'},
 UUID('016beef3-e6aa-3aec-a505-ff7b86ce6b80'): {'aliases': {'irish'},
  'name': 'irish'},
 UUID('019373c3-d920-36ad-91f3-edd64e5f010c'): {'aliases': {'opera'},
  'name': 'opera'},
 UUID('02cc0e4e-f78f-368e-bb05-8f54cb45988b'): {'aliases': {'ESC'},
  'name': 'esc'},
 UUID('02cc87ec-c6e3-37a2-9085-e16e94d4f6c9'): {'aliases': {'australian'},
  'name': 'australian'},
 UUID('04a86fb6-3ee3-3748-a795-0fe0f78ea89b'): {'aliases': {'Favorite'},
  'name': 'favorite'},
 UUID('0668030d-ec46-3166-bbca-5a988e3fa01d'): {'aliases': {'Serbia'},
  'name': 'serbia'},
 UUID('072067b5-c692-382b-b180-f9870507697f'): {'aliases': {'france'},
  'name': 'france'},
 UUID('09c36219-7d7c-3f6d-bbc6-c0ca20320b69'): {'aliases': {'post-anal experience'},
  'name': 'post-anal-experience'},
 UUID('0c515239-3dbf-38c6-a379-994a2b5fca98'): {'aliases': {

In [63]:
pd.DataFrame.from_dict(tags_list, orient='index').to_csv('tags-' + TODAY + 
                                                         '.csv', 
                                                         index_label='id')

In [64]:
pd.DataFrame(tags_connections).to_csv('tags-connections-' + TODAY + '.csv', 
                                      index=False)